Todo
Create streamlit app
Rag fusion 
Search other rag techniques that would make your output better 
How do you run all your code without creating a new index every time.


In [101]:
#import standard libraries
import os
from typing import List
import json
import logging 

In [102]:
#import Langchain Lbraries
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain.tools import tool
from langchain.agents import create_react_agent
from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain. retrievers.multi_query import MultiQueryRetriever
from langchain_core.messages.human import HumanMessage
from langchain.memory import ConversationBufferMemory



In [103]:
#import pinecone libraries
import pinecone
from langchain_pinecone import PineconeVectorStore

In [104]:
#import utilis
from airtable import AirTable

In [105]:
#import secret keys
from dotenv import load_dotenv 
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pine_cone_api = os.getenv("PINECONE_API_KEY")

In [106]:
#setup Langsmith for debugging
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__c4c7a9772ef046038636017b03b5f17b"

In [107]:
Langsmith_client = Client()

In [108]:
import prompts.prompts_2024_03_26 as prompts




In [9]:
airtable = AirTable()
airtable.post_data_to_table(data={"name_of_dev": "kuba",
                "model_name": "",
                "human_prompt": "",
                "global_context": "",
                "context_entrichment_agent_system_prompt": "",
                "enriched_prompt_output": "",
                "rag_extractor_agent_system_prompt": "",
                "rag_optimized_queries_output": "",
                "total_chunks_retrieved": "",
                "chunk_size": "",
                "knowledge_base_docs_list": "",
                "evaluator_agent_system_prompt": "",
                "reranked_rag_queries_output": "",
                "answer_agent_system_prompt": "",
                "final_answer_to_user": ""})



        To post data to the Airtable table.
        Args:
            data (dict): A dictionary with the format `{'column_name': data_to_post, ...}`.
        
        Keys for dictionary:
              {
                name_of_dev: "",
                model_name: "",
                human_prompt: "",
                global_context: "",
                context_entrichment_agent_system_prompt: "",
                enriched_prompt_output: "",
                rag_extractor_agent_system_prompt: "",
                rag_optimized_queries_output: "",
                total_chunks_retrieved: "",
                chunk_size: "",
                knowledge_base_docs_list: "",
                evaluator_agent_system_prompt: "",
                reranked_rag_queries_output: "",
                answer_agent_system_prompt: "",
                final_answer_to_user: ""
              }
        
data posted to Airtable successfully.


In [109]:
#initiaze large language models
llm  = ChatOpenAI(
    openai_api_key=openai_api_key, 
    model="gpt-4-turbo-preview", 
    temperature= 0.4
    )

In [110]:
#import global context file 
with open("Fastrise_global_context.txt", "r") as file:
     global_context = file.read()
    


COMPONENTS OF CONTEXT ENRICHMENT AGENT
global_context document 
tool 
prompt template
human input

Context Agent 

In [115]:
def Context_Enrichment_Agent(human_prompt: str):
   
    #import global context file 
    with open("Fastrise_global_context.txt", "r") as file:
     global_context = file.read()
    
    
    @tool
    def Enrichement_tool(human_prompt:str)-> str:
        "use this tool any time you get question about meetings, tasks, activities in a organization called Fastrise"
        global global_context
        return global_context

    tools = [Enrichement_tool]

    #create context_enrichment prompt template
    context_enrichment_template = prompts.context_enrichment_template
    context_prompt = PromptTemplate.from_template(template=context_enrichment_template, MessagePlaceholder=["(agent_scratchpad)"])
    
    #creating agent from the template
    context_agent = create_react_agent(
                            llm=llm, 
                           tools=tools, 
                           prompt=context_prompt)
    
   #invoke the agent
    context_agent_executor = AgentExecutor(
    agent=context_agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True, 
    )
    context_enriched_output = context_agent_executor.invoke({"input": human_prompt })
    Enriched_prompt = context_enriched_output["output"]
    
    return Enriched_prompt


In [180]:
human_prompt = "Given a list of tasks to be completed knowledgenow"

In [181]:
Enriched_prompt_output = Context_Enrichment_Agent(human_prompt=human_prompt)



> Entering new AgentExecutor chain...
Question: Given a list of tasks to be completed knowledgenow

Thought: To enrich the user's prompt effectively, I need to understand the specific tasks mentioned, their relevance to Fasterise's organizational goals, and any underlying context that might influence their prioritization or execution. This requires integrating knowledge about Fasterise's strategic objectives, the teams involved, and any recent developments or challenges that could impact these tasks.

Action: Enrichement_tool

Action Input: Given a list of tasks to be completed knowledgenow
### Mission and Vision
- *Mission and Vision*: Fasterise is centered around leveraging AI technology to amplify human potential, prioritizing ethical AI development that enriches human capabilities. The company envisions a future where AI solutions address societal needs and business challenges while augmenting the human potential and keeping it at the center of the system.
### Team Structure and 

In [137]:
Enriched_prompt_output

"In their meeting, Alejo and Bartek discussed strategic initiatives and projects at Fasterise, focusing on leveraging AI technology to amplify human potential and address societal needs. They covered the roles and contributions of team members, including their own in operations, sales, development, engineering, and prompt engineering with AI agents. They highlighted completed projects like the Ski Chalet Concierge, Scheduler Assistant, and Lead Qualifier, and current projects such as the Zoom Transcriber to Pinecone Integration and KnowledgeNow. The discussion also touched on the company's mission, vision, culture of innovation, profit-sharing, and the ethical, human-centered approach to AI development. This aligns with Fasterise's goals of ethical AI development, enhancing operational efficiency, and exploring AI's transformative societal impact."

RAG EXTRACTOR AGENT


In [182]:
def Rag_extractor_agent(Enriched_prompt_output:str) -> list:

#Creating a class for my output parser
    class sub_query(BaseModel):

        sub_queries: list = Field(description="""The sub queries extracted from the context_enriched_prompt""")


    parser = JsonOutputParser(pydantic_object=sub_query)


#defining the Rag_extractor tool
    @tool
    def Rag_extractor(Enriched_prompt_output:str)-> str:
        "use this tool any time you need to access context-enriched prompt for creating the sub-queries"
        return Enriched_prompt_output

    tools = [Rag_extractor]

    #create rag_extractor prompt template
    rag_extractor_template = prompts.rag_template
    rag_extractor_prompt = PromptTemplate.from_template(template=rag_extractor_template, MessagePlaceholder=["(agent_scratchpad)"])
    #creating agent from the template
    rag_extractor_agent = create_react_agent(llm=llm, 
                           tools=tools, 
                           prompt=rag_extractor_prompt)
    #invoke the agent
    rag_extractor_agent_executor = AgentExecutor(
    agent=rag_extractor_agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True, 
    )
    rag_optimized_queries_output = rag_extractor_agent_executor.invoke({"input":human_prompt,
                       "chat_history": [], 
                       "format_instructions":parser.get_format_instructions() })
    
    rag_optimized_queries = rag_optimized_queries_output["output"]

    Rag_query= rag_optimized_queries


    Rag_query_cleaned = Rag_query.replace('```json\n', '').replace('\n```', '')

    # Convert the JSON string into a dictionary
    Rag_query_dict = json.loads(Rag_query_cleaned)

    # Extract the sub-queries
    sub_queries = Rag_query_dict["sub_queries"]

    return sub_queries



In [185]:
sub_queries = Rag_extractor_agent(Enriched_prompt_output)



> Entering new AgentExecutor chain...


```json
{
  "sub_queries": [
    "List all current tasks assigned within the KnowledgeNow project",
    "Identify the deadlines for each task in the KnowledgeNow project",
    "Determine the current progress status of each KnowledgeNow task",
    "Extract any recent updates or changes made to tasks in the KnowledgeNow project",
    "Identify team members responsible for each task in the KnowledgeNow project"
  ]
}
```Invalid Format: Missing 'Action:' after 'Thought:```json
{
  "sub_queries": [
    "List all current tasks assigned within the KnowledgeNow project",
    "Identify the deadlines for each task in the KnowledgeNow project",
    "Determine the current progress status of each KnowledgeNow task",
    "Extract any recent updates or changes made to tasks in the KnowledgeNow project",
    "Identify team members responsible for each task in the KnowledgeNow project"
  ]
}
```Invalid Format: Missing 'Action:' after 'Thought:Action: Rag_extractor
Thought: I now know the final answer
F

In [186]:
sub_query_texts = [sub_query for sub_query in sub_queries]
print(sub_query_texts)


['List all current tasks assigned within the KnowledgeNow project', 'Identify the deadlines for each task in the KnowledgeNow project', 'Determine the current progress status of each KnowledgeNow task', 'Extract any recent updates or changes made to tasks in the KnowledgeNow project', 'Identify team members responsible for each task in the KnowledgeNow project']


In [192]:
sub_query_1 = sub_query_texts[0]
sub_query_2 = sub_query_texts[1]
sub_query_3 = sub_query_texts[2]
sub_query_4 = sub_query_texts[3]
print(sub_query_1)
print(sub_query_2)
print(sub_query_3)
print(sub_query_4)

List all current tasks assigned within the KnowledgeNow project
Identify the deadlines for each task in the KnowledgeNow project
Determine the current progress status of each KnowledgeNow task
Extract any recent updates or changes made to tasks in the KnowledgeNow project


RAG 

In [131]:
#loading company docs
loader = DirectoryLoader("data",
                    glob="**/*.txt",
                    loader_cls= TextLoader)
text_documents = loader.load()

len(text_documents)
    

 


2

In [208]:
#split documents into chunks
text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type="percentile")
text_chunk = text_splitter.split_documents(text_documents)

print("The length of the chunk:", len(text_chunk))

The length of the chunk: 77


In [171]:
#text_splitter = RecursiveCharacterTextSplitter(
        #chunk_size= 400,
        #chunk_overlap=0,
        #length_function=len
    #)
#text_chunk = text_splitter.split_documents(text_documents)

#print("The length of the chunk:", len(text_chunk))

The length of the chunk: 367


In [209]:
for i, _ in enumerate(text_chunk):
    print(f"chunk {i} : {text_chunk[i]}")


chunk 0 : page_content="\n01:09\nAlejo\nOkay. Hi, see you all again. 01:17\nBartek\nWaiting for us or it's only three of us here? Hi guys, I'm hearing you right now. I just had a dinner, so excuse me, eating. Aleo, I think you have something with your audio. I can't hear you. Aleo, you're not here. 02:49\nAlejo\nOkay, I guess no headphones it is. Hey you guys. 02:53\nBartek\nYeah, sounds good. 02:54\nAlejo\nCool. 02:55\nBartek\nHi again. 02:57\nAlejo\nHey, did you say something? Is there something you wanted to share? 03:05\nBartek\nI just wanted to share that I have great. Dinner or lunch?" metadata={'source': 'data\\2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt'}
chunk 1 : page_content="Lunch. I messed that up every time. Lunch. 03:13\nKuba\nHi guys. 03:15\nAlejo\nHey Kuba. 03:23\nKuba\nLet'S see how you doing, bro? 03:35\nAlejo\n1 second, you guys, thank you guys for sharing your update on Friday. It makes me very happy to see progress on what you get to work on or what you w

In [210]:
#creating embeddings

model_name = "text-embedding-3-small"
def data_embedding():
    embeddings = OpenAIEmbeddings(model=model_name)
    return embeddings

embeddings = data_embedding()

In [211]:
#initializing Vectorstore
Pinecone_api_key = os.getenv("PINECONE_API_KEY")
Your_env = "gcp-starter"
index_name = "knowledgenow"

In [212]:
#Adding embeddings into the vectorstore
vectorstore = PineconeVectorStore.from_documents(text_chunk, embeddings, index_name=index_name)

In [213]:
retriver = vectorstore.as_retriever(search_kwargs={'k':25})

multi_llm = ChatOpenAI(temperature= 0.5)

#Adding mutli query retriver
retriever = MultiQueryRetriever.from_llm( retriever=retriver, llm=multi_llm
                )

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#retrieval chain
qa = RetrievalQAWithSourcesChain.from_chain_type(
llm=multi_llm, 
chain_type="stuff", 
retriever=retriever)




KNOWLEDGEBASE QUERY AGENT

In [214]:
sub_query = sub_query_1

In [215]:

@tool
def knowledgebase(retrived_doc:str)-> str:
    """Use this tool when you want extract information from the Knowledgenow database"""
    result_str = qa.invoke(sub_query)
    return result_str

tools = [knowledgebase]

 #create rag_extractor prompt template
knowledgebase_query_template = prompts.Knowledgebase_template
knowledgebase_query_prompt = PromptTemplate.from_template(template=knowledgebase_query_template, MessagePlaceholder=["(agent_scratchpad)"])
knowledgebase_query_agent = create_react_agent(llm=llm, 
                           tools=tools, 
                           prompt=knowledgebase_query_prompt)
knowledgebase_query_agent_executor = AgentExecutor(agent=knowledgebase_query_agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=8)




In [216]:
Knowledgebase_agent_output= knowledgebase_query_agent_executor.invoke({"sub_query": sub_query})





> Entering new AgentExecutor chain...
I need to query the KnowledgeNow database to find all the current tasks assigned within the KnowledgeNow project. This will require extracting specific information about tasks, including their descriptions, assignees, and deadlines if available.

Action: knowledgebase
Action Input: List all current tasks assigned within the KnowledgeNow project


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the current tasks being worked on in the KnowledgeNow project?', '2. Can you provide a list of tasks currently assigned in the KnowledgeNow project?', '3. What tasks are currently underway in the KnowledgeNow project?']


{'question': 'List all current tasks assigned within the KnowledgeNow project', 'answer': "Tasks assigned within the KnowledgeNow project are:\n1. Share intermediate and advanced prompt engineering resources with Kuba\n2. Draft prompts for context enrichment agent, RAG extractor agent, evaluator agent, and answer formatting agent in relation to Knowledge Now MVP\n3. Review Bartek's shared prompt engineering resources\n4. Work on draft prompts provided by Bartek for Knowledge Now MVP once received\n5. Showcase integrating Pine Cone Vector Database and OpenAI embeddings to generate responses from meeting transcriptions\n6. Develop automation services to streamline the project management process\n7. Familiarize with markdown syntax for better documentation\n\n", 'sources': 'data\\2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt, data\\2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt'}I now know the final answer.

Final Answer: The current tasks assigned within the KnowledgeNow pro

In [217]:
sub_query = sub_query_2

In [218]:
Knowledgebase_agent_output_2= knowledgebase_query_agent_executor.invoke({"sub_query": sub_query_2})



> Entering new AgentExecutor chain...
I need to query the KnowledgeNow database to find specific deadlines for each task within the KnowledgeNow project. This will involve extracting dates and associated tasks from the project documentation stored within the database.

Action: knowledgebase
Action Input: Identify the deadlines for each task in the KnowledgeNow project

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the specific deadlines associated with each task within the KnowledgeNow project?', '2. Can you provide a breakdown of the task deadlines within the KnowledgeNow project?', '3. How are the deadlines distributed across the various tasks in the KnowledgeNow project?']


{'question': 'Identify the deadlines for each task in the KnowledgeNow project', 'answer': 'The deadlines for each task in the KnowledgeNow project were not explicitly mentioned in the provided content.\n', 'sources': ''}I now know the final answer

Final Answer: The deadlines for each task in the KnowledgeNow project were not explicitly mentioned in the provided content.

> Finished chain.


In [219]:
sub_query = sub_query_3

In [220]:
Knowledgebase_agent_output_3 = knowledgebase_query_agent_executor.invoke({"sub_query": sub_query_3})



> Entering new AgentExecutor chain...
I need to query the KnowledgeNow database to find the current progress status of each task within the KnowledgeNow system. This involves extracting specific details regarding the status of tasks, such as whether they are in progress, completed, or pending, and any relevant metrics or updates that indicate their current state.

Action: knowledgebase
Action Input: What is the current progress status of each KnowledgeNow task?


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the current status update for all tasks in KnowledgeNow?', '2. Can you provide an update on the progress of each task within KnowledgeNow?', '3. How can I check the current progress status of individual tasks in KnowledgeNow?']


{'question': 'Determine the current progress status of each KnowledgeNow task', 'answer': 'The progress status of each KnowledgeNow task is not explicitly mentioned in the provided content.\n', 'sources': ''}I now know the final answer

Final Answer: The current progress status of each KnowledgeNow task is not explicitly mentioned in the provided content.

> Finished chain.


In [221]:
sub_query = sub_query_4

In [222]:
knowledgebase_agent_output_4 = knowledgebase_query_agent_executor.invoke({"sub_query": sub_query_4})



> Entering new AgentExecutor chain...
I need to query the KnowledgeNow database to find the most recent updates or changes made to tasks within the KnowledgeNow project. This will involve searching for documents or entries that detail task modifications, updates, or any form of change log related to the project tasks.

Action: knowledgebase
Action Input: Recent updates or changes made to tasks in the KnowledgeNow project


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the most recent modifications or updates to tasks within the KnowledgeNow project?', '2. Can you provide me with the latest changes or updates made to tasks in the KnowledgeNow project?', '3. Have there been any recent updates or revisions to tasks in the KnowledgeNow project that I should be aware of?']


{'question': 'Extract any recent updates or changes made to tasks in the KnowledgeNow project', 'answer': "Recent updates in the KnowledgeNow project include the implementation of automation services to streamline project management processes and the use of 'Knowledgenow' for task management, generating a list of tasks for team members after each meeting. There is also a focus on enhancing content creation and idea generation within the project. Additionally, there are ongoing discussions about integrating various AI agents to improve system performance and efficiency.\n", 'sources': 'data\\2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt, data\\2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt'}I now know the final answer

Final Answer: Recent updates in the KnowledgeNow project include the implementation of automation services to streamline project management processes and the use of 'Knowledgenow' for task management, generating a list of tasks for team members after each mee

In [57]:

Knowledgebase_answer_1 = Knowledgebase_agent_output["output"]
Knowledgebase_answer_2 = Knowledgebase_agent_output_2["output"]
Knowledgebase_answer_3= Knowledgebase_agent_output_3["output"]
Knowledgebase_answer_4= knowledgebase_agent_output_4["output"]
Knowledgebase_answer_5= knowledgebase_agent_output_5["output"]

combined_output_list = [Knowledgebase_answer_1, Knowledgebase_answer_2, Knowledgebase_answer_3, Knowledgebase_answer_4, Knowledgebase_answer_5]
print(combined_output_list)

['During the meeting, Alejo and Bartek were involved in discussing key topics such as the development of a multi-agent system for the Knowledge Now project. This included focusing on agents like context enrichment, rag extractor, evaluator, QA, supervisor, merger, and answer formatting agents. They also emphasized the importance of feedback loops for prompt generation and the need for a swarm system to improve prompt quality. However, the provided information does not explicitly mention specific action items assigned to Alejo and Bartek.', 'The key topics discussed by Alejo and Bartek in the meeting include the development of agents for the Knowledge Now system, such as context enrichment, query agent, evaluation agent, and generation agent. They also discussed the need for a feedback loop for prompt generation and the importance of simplicity and programmatic processes. Additionally, they touched on integrating Notion with Relevance AI for content automation and the importance of real

Evaluator Agent

In [58]:
#creating a custom tool
@tool
def Vectormatches(query:str)-> str:
    """
    Use this tool when you want to return vector match results for the vectordatabase"""
    global combined_output_list
    return combined_output_list

In [59]:
tools = [Vectormatches]

In [60]:
evaluator_template = prompts.evaluator_template

evaluator_prompt = PromptTemplate.from_template(template=evaluator_template, MessagePlaceholder=["(agent_scratchpad)"])

In [61]:

evaluator_agent = create_react_agent(llm=llm, tools=tools, prompt=evaluator_prompt)

In [62]:
evaluator_agent_executor = AgentExecutor(agent=evaluator_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)

In [68]:
Evalutor_output=evaluator_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt_output,
        "Vector_matches": combined_output_list})



> Entering new AgentExecutor chain...
Given the enriched prompt and the vector matches, it's clear that Alejo and Bartek's discussion focused heavily on the technical development and strategic planning of AI technologies within the KnowledgeNow project and potentially other projects at Fasterise. Their conversation spanned a range of topics crucial for the integration and enhancement of AI capabilities, emphasizing the development of a multi-agent system, the importance of feedback loops for prompt generation, and the integration of advanced technologies like the Pine Cone Vector Database and OpenAI embeddings.

The discussion points indicate a deep dive into the operational and technical specifics necessary for advancing Fasterise's mission of leveraging AI to amplify human potential. The emphasis on multi-agent systems, including specific roles like context enrichment, RAG extractor, evaluator, and answer formatting agents, aligns with Fasterise's focus on creating sophisticated, e

In [69]:
Evaluator_agent_output=Evalutor_output["output"]
print(Evaluator_agent_output)

Alejo and Bartek discussed several key topics in their meeting, focusing on the development and strategic planning of AI technologies within the KnowledgeNow project and potentially other initiatives at Fasterise. Their conversation covered the technical development of a multi-agent system, including specific agent roles like context enrichment and RAG extractor, and the strategic application of AI across various operational areas, such as content automation and real estate. The emphasis on feedback loops, the integration of advanced technologies like the Pine Cone Vector Database and OpenAI embeddings, and the potential for AI-driven social media strategies reflect a deep commitment to leveraging AI to amplify human potential, aligning with Fasterise's mission and vision.


ANSWER AGENT

In [70]:
#creating a custom tool
@tool
def Evaluator_result(query:str)-> str:
    """
    Use this tool anytime you need to return  Re-ranked and evaluated data results from the Evaluator assistant """
    global Evaluator_agent_output
    return Evaluator_agent_output

In [71]:
Answer_template = prompts.Answer_template

Answer_prompt = PromptTemplate.from_template(template=Answer_template, MessagePlaceholder=["(agent_scratchpad)"])

In [72]:
Answer_agent = create_react_agent(llm=llm, tools=tools, prompt=Answer_prompt)

In [73]:
Answer_agent_executor = AgentExecutor(agent=Answer_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)

In [76]:
Answer_output=Answer_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt_output})



> Entering new AgentExecutor chain...
To address this query, we need to delve into the specifics of what Alejo and Bartek, key figures at Fasterise, might have discussed in their meeting, focusing on their roles and the company's strategic direction. Alejo, deeply involved in Operations, Sales, and Development with an emphasis on AI technologies including RAG, LLMs, and Multi-Agent Systems, and Bartek, leading engineering with a specialization in Prompt Engineering and AI Agents, are pivotal in steering Fasterise's innovative projects.

Given their expertise and the context of Fasterise's mission, it's plausible that their discussion revolved around several critical areas:

1. **Integration of AI Technologies in Current Projects**: They likely explored how to further incorporate AI technologies like RAG and LLMs into ongoing projects such as the Zoom Transcriber to Pinecone Integration and KnowledgeNow. This could involve technical discussions on optimizing these technologies for bet

In [77]:
Answer_agent_output = Answer_output["output"]
print(Answer_agent_output)

Agent stopped due to iteration limit or time limit.


In [142]:

#conversational memory 
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True

)
conversational_memory.chat_memory.messages
conversational_memory.chat_memory.add_user_message(input)
conversational_memory.chat_memory.add_ai_message(Answer_agent_output)

In [145]:

def memory2str(memory:ConversationBufferWindowMemory):
    messages = memory.chat_memory.messages
    memory_list = [
        f"Human: {mem.content}" for mem in messages 
    ]
    memory_str = "/n".join(memory_list)
    return memory_str

In [152]:
def chat(text:str):
    Answer_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": Enriched_prompt_output})
    conversational_memory.chat_memory.add_user_message(text)
    conversational_memory.chat_memory.add_ai_message(Answer_agent_output)
    return Answer_agent_output

In [153]:
chat({"input":"what are the things they said about the project knowledgenow"})



> Entering new AgentExecutor chain...
Given the enriched context and the focus areas of Alejo and Bartek at Fasterise, it's reasonable to infer that their meeting discussion revolved around the strategic integration of AI technologies within Fasterise's current and upcoming projects. Specifically, they likely deliberated on how to effectively apply RAG, LLMs, and Multi-Agent Systems to enhance the KnowledgeNow and Personalized Outreach System projects. This aligns with Fasterise's overarching mission of leveraging AI to amplify human potential while adhering to ethical AI development principles. Their conversation would have been a blend of technical planning, operational strategy, and possibly, the exploration of new AI applications that could further Fasterise's goals of operational efficiency, improved customer experience, and societal impact through AI innovation.

### Key Discussion Points:
- **Integration of AI Technologies**: Alejo and Bartek likely discussed the technical asp

: 